# Document Classification Project


In [36]:
# set up libraries 
import pandas as pd
import glob
import os
from bs4 import BeautifulSoup
import requests
from spacy.lang.en import English
from collections import Counter


In [3]:
docs = pd.read_csv("C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/Data2.csv")
docs.dropna(inplace=True)
print(docs.iloc[22][0])

https://counteriedreport.com/egyptian-military-kills-77-terrorists-in-northern-sinai-report/


In [4]:
# Get txt files from urls
for i in range(len(docs)):
    try:
        r = requests.get(docs.iloc[i][0], timeout=10)
        print(i, r)
        print(docs.iloc[i][0])
    except:
        print("Problem:", i)
        continue

    if r.ok:
        soup = BeautifulSoup(r.content, 'html.parser')
        with open("data/Reports/" + str(i) + ".txt", 'wt') as f:
            text = ""
            for element in soup.select('p'):
                # Reduce potential overfitting by not including parts of the document that repeat themselves regularly and aren't relevant
                # to the actual text
                if  "View/Print Poster" in element.text:
                    continue
                elif element.text == "If you prefer a different language, you may select one from the full list by navigating to the Select Language menu at the top of this site.":
                    continue
                elif element.text == "Rewards for Justice":
                    continue
                elif element.text == "The most important reasons to stop a terrorist are all around you.":
                    continue
                elif element.text == "Stop a terrorist. Save lives.":
                    continue
                elif element.text == "Up to $10 Million Reward":
                    continue
                elif element.text == "{{region_detail.text}}":
                    continue
                elif element.text == "{{person_detail.text}}":
                    continue
                elif element.text == "{{organization_detail.text}}":
                    continue
                elif element.text == "{{series.description | limitTo:140}}":
                    continue
                elif element.text == "Up to $5 Million Reward":
                    continue
                elif len(element.text.split()) == 1:
                    continue
                text += element.text + "\n"
            try:
                f.write(text)
            except:
                print("Write problem:", i)
                continue
        


0 <Response [200]>
https://issafrica.org/iss-today/time-to-tackle-violence-against-women-in-mali
1 <Response [200]>
http://www.understandingwar.org/backgrounder/russia-review-putins-%E2%80%9Cpeacekeepers%E2%80%9D-will-support-russian-wars
2 <Response [200]>
http://www.understandingwar.org/backgrounder/russian-president-putin-wins-upset-victory-nagorno-karabakh
3 <Response [200]>
http://www.understandingwar.org/backgrounder/warning-intelligence-update-iran-increases-pressure-us-forces-iraq
Write problem: 3
4 <Response [200]>
http://www.understandingwar.org/backgrounder/russia-turkey-competition-escalates-across-theaters
5 <Response [200]>
https://www.middleeasteye.net/news/uk-foreign-aid-cut-impact-middle-east-yemen-syria
6 <Response [200]>
https://www.middleeasteye.net/opinion/israel-palestine-deal-century-biden-end
7 <Response [200]>
https://www.middleeasteye.net/news/israel-saudi-arabia-netanyahu-mbs-secret-meeting-meaning
8 <Response [200]>
https://www.defenseindustrydaily.com/f18-h

In [4]:
# Get data
df = pd.DataFrame(columns=['Text', 'Label'])
main_path = r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/'
end_path = ['Analytical_Report', 'Incident_Report', 'Situation_Report']
for end in end_path:
    path = main_path + end
    all_files = glob.glob(os.path.join(path, "*.txt")) 
    # print(all_files[0])
    data = [[f, end] for f in all_files]
    # print(pd.DataFrame(data, columns=['Text', 'Label']))
    df = df.append(pd.DataFrame(data, columns=['Text', 'Label']))
# print(df)

main_path= r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/Reports'
files = glob.glob(os.path.join(main_path, '*.txt'))
data2 = [[f, docs.iloc[i][1]] for i, f in enumerate(files)]
df = df.append(pd.DataFrame(data2, columns=['Text', 'Label']))
print(df)

                                                  Text              Label
0    C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
1    C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
2    C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
3    C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
4    C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
..                                                 ...                ...
172  C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical Report
173  C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical Report
174  C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical Report
175  C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical Report
176  C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical Report

[334 rows x 2 columns]


In [5]:
# Convert 'label' column to categories
df["Label"] = df["Label"].replace({'Analytical Report' : 0, 'Analytical_Report' : 0, 'Incident Report' : 1, 'Incident_Report' : 1, 'Profile Report' : 2, 'Profile_Report' : 2, 'Report' : 2, 'Situation Report' : 3, 'Situation_Report' : 3 })
df["Label"] = df["Label"].astype('category')
df["Label"]

0      0
1      0
2      0
3      0
4      0
      ..
172    0
173    0
174    0
175    0
176    0
Name: Label, Length: 334, dtype: category
Categories (4, int64): [0, 1, 2, 3]

In [6]:
# How may documents in each category?
df.groupby(['Label']).count()

,Text
Label,
0,47
1,88
2,100
3,99


Now it's time to add some potentially useful features to the dataset, by looking at what we already know about the documents and making some assumptions. Then we can perform EDA to check how relevant these features are before running models using only the most important features.

In [62]:
# Total number of words
word_counts, unique_word_counts, vocabs_per_wc = [], [], []

common_words = [[],[],[],[],[]]
# create spacy object
nlp = English()

# go through each row and read in the text
for idx, row in df.iterrows():
    # read in the text
    try:   
        corpus = open(row[0], encoding="utf8").read()
    except:
        corpus = open(row[0], encoding="latin1").read()
    doc = nlp(corpus)
    list_of_words = [token.text for token in doc if not token.is_punct] # get the words and numbers (not punctiation)

    word_count = len(list_of_words) # Get the word count
    unique_word_count = len(set(list_of_words)) # Get the word count of unique words
    vocab_per_wc = unique_word_count / word_count # Get the proportion of words that are unique
    
    # get a list of words that are not stop words or punctuation
    list_of_non_stop_words = [token.text for token in doc if not token.is_stop and not token.is_punct and token.text != "\n"]
    word_freq = Counter(list_of_non_stop_words) # count the frequency of each word
    for i in range(5):
        common = word_freq.most_common(i+1) # Get the ith most common words
        common_words[i].append([tuple_[0] for tuple_ in common]) # get just the string and not the number
    
    word_counts.append(word_count)
    unique_word_counts.append(unique_word_count)
    vocabs_per_wc.append(vocab_per_wc)

df['Wordcount'] = word_counts
df['Vocabulary'] = unique_word_counts
df['Vocabulary_per_wordcount'] = vocabs_per_wc
df['common_word'] = common_words[0]
df['common_words_2'] = common_words[1]
df['common_words_3'] = common_words[2]
df['common_words_4'] = common_words[3]
df['common_words_5'] = common_words[4]
# print(df['Wordcount'])
# print(df['Vocabulary'])
# print(df["Vocabulary_per_wordcount"])
print(df['common_word'])

0      [retirement]
1             [tax]
2         [Lebanon]
3         [Estonia]
4           [water]
           ...     
172       [Haqqani]
173          [Harb]
174          [U.S.]
175      [aircraft]
176          [ISIS]
Name: common_word, Length: 334, dtype: object


In [52]:
common_words = [[],[],[],[],[]]
common_words[0].append("1")
common_words[0].append()
common_words

[['1', '2'], [], [], [], []]